In [141]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.linear_model import LogisticRegression

In [142]:
dftestt = pd.read_csv('/content/titanic_reserved.csv')
dftestt

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.5500,NaN,S,NaN
1,1,"Snyder, Mr. John Pillsbury",male,24.0000,1,0,21228,82.2667,B45,S,"Minneapolis, MN"
2,2,"Ashby, Mr. John",male,57.0000,0,0,244346,13.0000,NaN,S,"West Hoboken, NJ"
3,3,"Rosblom, Mr. Viktor Richard",male,18.0000,1,1,370129,20.2125,NaN,S,NaN
4,2,"Doling, Miss. Elsie",female,18.0000,0,1,231919,23.0000,NaN,S,Southampton
...,...,...,...,...,...,...,...,...,...,...,...
323,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6000,NaN,C,"Philadelphia, PA"
324,1,"Romaine, Mr. Charles Hallace (""Mr C Rolmane"")",male,45.0000,0,0,111428,26.5500,NaN,S,"New York, NY"
325,1,"Meyer, Mrs. Edgar Joseph (Leila Saks)",female,NaN,1,0,PC 17604,82.1708,NaN,C,"New York, NY"
326,2,"Caldwell, Master. Alden Gates",male,0.8333,0,2,248738,29.0000,NaN,S,"Bangkok, Thailand / Roseville, IL"


In [143]:
dfn = pd.read_csv('/content/titanic_train.csv')
dfn

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,3,1,"Smyth, Miss. Julia",female,NaN,0,0,335432,7.7333,NaN,Q,NaN
1,3,1,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,NaN,Q,"Co Clare, Ireland Washington, DC"
2,3,1,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,0,0,2688,7.2292,NaN,C,NaN
3,3,0,"Vovk, Mr. Janko",male,22.0,0,0,349252,7.8958,NaN,S,NaN
4,3,0,"de Pelsmaeker, Mr. Alfons",male,16.0,0,0,345778,9.5000,NaN,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
976,3,1,"Howard, Miss. May Elizabeth",female,NaN,0,0,A. 2. 39186,8.0500,NaN,S,NaN
977,3,1,"Dorking, Mr. Edward Arthur",male,19.0,0,0,A/5. 10482,8.0500,NaN,S,"England Oglesby, IL"
978,2,0,"Gillespie, Mr. William Henry",male,34.0,0,0,12233,13.0000,NaN,S,"Vancouver, BC"
979,3,0,"Barton, Mr. David John",male,22.0,0,0,324669,8.0500,NaN,S,"England New York, NY"


In [144]:
df = dfn.copy()
dftest = dftestt.copy()
df['fam_size'] = df['parch'] + df['sibsp']
dftest['fam_size'] = dftest['parch'] + dftest['sibsp']

df.drop(['home.dest', 'cabin', 'ticket', 'sibsp', 'parch'], axis = 1, inplace = True)
dftest.drop(['home.dest', 'cabin', 'ticket', 'sibsp', 'parch'], axis = 1, inplace = True)

mr = ['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt']
mrs = ['Countess', 'Dona']
df['hon'] = None
dftest['hon'] = None
for i in range(df.shape[0]):
  a = df.loc[i, 'name'].split()
  a = [x[:-1] for x in a if '.' in x ][0]
  df.loc[i, 'hon'] = a

for i in range(dftest.shape[0]):
  a = dftest.loc[i, 'name'].split()
  a = [x[:-1] for x in a if '.' in x ][0]
  dftest.loc[i, 'hon'] = a

#print(df['name'].unique())

df['hon'] = ['Mr' if x in mr else 'Mrs' if x in mrs else x for x in df['hon']]
df['age'] = df.groupby('hon')['age'].transform(lambda x: x.fillna(x.mode()[0]))
dftest['hon'] = ['Mr' if x in mr else 'Mrs' if x in mrs else x for x in dftest['hon']]
dftest['age'] = dftest.groupby('hon')['age'].transform(lambda x: x.fillna(x.mode()[0]))

#mean 0.7516778523489933
#mode 0.7567567567567568

df.drop(['name'], axis = 1, inplace = True)
dftest.drop(['name'], axis = 1, inplace = True)

df = pd.get_dummies(df, drop_first = True)
dftest = pd.get_dummies(dftest, drop_first = True)

X = df.drop(['survived'], axis = 1)
Y = df['survived']
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 21, stratify = df['survived'])
LR = LogisticRegression(max_iter = 1000, random_state = 21).fit(x_train, y_train)
y_pred = LR.predict(x_test)

train_columns = x_train.columns

for col in train_columns:
    if col not in dftest.columns:
        dftest[col] = 0

dftest = dftest[train_columns]

f1_score(y_test, y_pred)

0.7567567567567568

In [145]:
LR = LogisticRegression(max_iter=1000, random_state=21).fit(X, Y)
y_pred = LR.predict(dftest)
y_pred


array([0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,